In [ ]:
# Import pandas
import pandas as pd

# Read in white wine data
white = pd.read_csv('C:/Users/Usuario/Downloads\winequality-white.csv', sep=';')

# Read in red wine data
red = pd.read_csv('C:/Users/Usuario/Downloads\winequality-red.csv', sep=';')

In [ ]:
# Add `type` column to `red` with value 1
red['type'] = 1

# Add `type` column to `white` with value 0
white['type'] = 0

# Append `white` to `red`
#wines = red.append(white, ignore_index=True)
wines = pd.concat([red, white], ignore_index=True)

In [ ]:
wines

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,0
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,0
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,0
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,0


In [ ]:
X=wines.iloc[:,0:12]

In [ ]:
X=wines.drop('quality',axis=1)

In [ ]:
# Isolate target labels
#Y = wines.quality
import numpy as np
Y=np.ravel(wines.quality)

In [ ]:
# Import `StandardScaler` from `sklearn.preprocessing`
from sklearn.preprocessing import StandardScaler

# Scale
X_train=StandardScaler().fit_transform(X)

In [ ]:
# Import `Sequential` from `keras.models`
from keras.models import Sequential

# Import `Dense` from `keras.layers`
from keras.layers import Dense

import numpy as np
from sklearn.model_selection import StratifiedKFold

seed = 7
np.random.seed(seed)

test = np.random.choice(X_train.shape[0], size=int(X_train.shape[0]*0.2), replace=False)
mse_vals = []
mae_vals = []

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
for train, val in kfold.split(X, Y):
    model = Sequential()
    model.add(Dense(64, input_dim=12, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    model.fit(X_train[train], Y[train], epochs=10,verbose=0)
    mse_aux_value, mae_aux_value=model.evaluate(X_train[val], Y[val], verbose=0)
    mse_vals.append(mse_aux_value)
    mae_vals.append(mae_aux_value)

mse_val_value = np.mean(mse_vals)
mae_val_value = np.mean(mae_vals)

#En caso de haber guardado un modelo...
y_test_pred = model.predict(X_train[test])
mse_test_value, mae_test_value=model.evaluate(X_train[test], Y[test], verbose=0)

print(f'Validation MSE: {mse_val_value:.5f}')
print(f'Validation MAE: {mae_val_value:.5f}')
print(f'Test MSE: {mse_test_value:.5f}')
print(f'Test MAE: {mae_test_value:.5f}')

41/41 [==============================] - 0s 3ms/step
Validation MSE: 0.51283
Validation MAE: 0.55232
Test MSE: 0.46977
Test MAE: 0.52988


In [ ]:
from sklearn.metrics import r2_score
r2_score(Y[test],y_test_pred)

0.37186213149241143

In [ ]:
datos = {
    'Tipo de capas': ['Dense'],
    'Tamaño capa de entrada': [64],
    'Épocas': [10],
    'Función de activación': "ReLU",
    'MSE': mse_test_value,
    'MAE': mae_test_value,
    'R2': r2_score(Y[test],y_test_pred)
}

df = pd.DataFrame(datos)
print(df)

  Tipo de capas  Tamaño capa de entrada  Épocas Función de activación  \
0         Dense                      64      10                  ReLU   

        MSE       MAE        R2  
0  0.469774  0.529881  0.371862  


### Pruebas con mayor número de épocas

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

epocas = [20, 30, 40, 50]

for i in range(len(epocas)):
    for train, val in kfold.split(X, Y):
        model = Sequential()
        model.add(Dense(64, input_dim=12, activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
        model.fit(X_train[train], Y[train], epochs=epocas[i],verbose=0)
        mse_aux_value, mae_aux_value=model.evaluate(X_train[val], Y[val], verbose=0)
        mse_vals.append(mse_aux_value)
        mae_vals.append(mae_aux_value)

    mse_val_value = np.mean(mse_vals)
    mae_val_value = np.mean(mae_vals)

#En caso de haber guardado un modelo...
    y_test_pred = model.predict(X_train[test])
    mse_test_value, mae_test_value=model.evaluate(X_train[test], Y[test], verbose=0)

    datos = {
    'Tipo de capas': 'Dense',
    'Tamaño capa de entrada': 64,
    'Épocas': epocas[i],
    'Función de activación': "ReLU",
    'MSE': mse_test_value,
    'MAE': mae_test_value,
    'R2': r2_score(Y[test],y_test_pred)
    }

    datos = pd.DataFrame([datos])
    df = pd.concat([df, datos], ignore_index=True)

print(df)

41/41 [==============================] - 0s 3ms/step
  Tipo de capas  Tamaño capa de entrada  Épocas Función de activación  \
0         Dense                      64      10                  ReLU   
1         Dense                      64      20                  ReLU   
2         Dense                      64      30                  ReLU   
3         Dense                      64      40                  ReLU   
4         Dense                      64      50                  ReLU   

        MSE       MAE        R2  
0  0.469774  0.529881  0.371862  
1  0.439342  0.516003  0.412554  
2  0.448071  0.516015  0.400882  
3  0.423551  0.502714  0.433667  
4  0.445968  0.512089  0.403693  


A mayor número de épocas se obtienen mejores resultados, el rendimiento aumenta considerablemente y así mismo los errores y pérdidas disminuyen. Aún así parece que el crecimiento no es ilimitado, parece ser que se alcanza una zona de convergencia en la que el rendimiento no aumenta más allá de las 50 épocas más o menos.

### Pruebas con mayor número de capas

In [ ]:
# Import `Sequential` from `keras.models`
from keras import Sequential

# Import `Dense` from `keras.layers`
from keras.layers import Dense

import numpy as np
from sklearn.model_selection import StratifiedKFold

seed = 7
np.random.seed(seed)

test = np.random.choice(X_train.shape[0], size=int(X_train.shape[0]*0.2), replace=False)
mse_vals = []
mae_vals = []

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
for train, val in kfold.split(X, Y):
    model = Sequential()
    model.add(Dense(64, input_dim=12, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    model.fit(X_train[train], Y[train], epochs=10,verbose=0)
    mse_aux_value, mae_aux_value=model.evaluate(X_train[val], Y[val], verbose=0)
    mse_vals.append(mse_aux_value)
    mae_vals.append(mae_aux_value)

mse_val_value = np.mean(mse_vals)
mae_val_value = np.mean(mae_vals)

#En caso de haber guardado un modelo...
y_test_pred = model.predict(X_train[test])
mse_test_value, mae_test_value=model.evaluate(X_train[test], Y[test], verbose=0)

print(f'Validation MSE: {mse_val_value:.5f}')
print(f'Validation MAE: {mae_val_value:.5f}')
print(f'Test MSE: {mse_test_value:.5f}')
print(f'Test MAE: {mae_test_value:.5f}')

from sklearn.metrics import r2_score
print(f'R2 score: {r2_score(Y[test],y_test_pred):.5f}')

41/41 [==============================] - 0s 3ms/step
Validation MSE: 0.54149
Validation MAE: 0.57331
Test MSE: 0.50935
Test MAE: 0.57093
R2 score: 0.31894


In [ ]:
datos_2 = {
    'Tipo de capas': ['Dense'],
    'Tamaño capa de entrada': [64],
    'Número de capas ocultas': [1],
    'Función de activación': "ReLU",
    'MSE': mse_test_value,
    'MAE': mae_test_value,
    'R2': r2_score(Y[test],y_test_pred)
}

df_2 = pd.DataFrame(datos_2)

In [ ]:
seed = 7
np.random.seed(seed)

test = np.random.choice(X_train.shape[0], size=int(X_train.shape[0]*0.2), replace=False)
mse_vals = []
mae_vals = []

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
for train, val in kfold.split(X, Y):
    model = Sequential()
    model.add(Dense(64, input_dim=12, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    model.fit(X_train[train], Y[train], epochs=10,verbose=0)
    mse_aux_value, mae_aux_value=model.evaluate(X_train[val], Y[val], verbose=0)
    mse_vals.append(mse_aux_value)
    mae_vals.append(mae_aux_value)

mse_val_value = np.mean(mse_vals)
mae_val_value = np.mean(mae_vals)

y_test_pred = model.predict(X_train[test])
mse_test_value, mae_test_value=model.evaluate(X_train[test], Y[test], verbose=0)

41/41 [==============================] - 0s 3ms/step


In [ ]:
datos_2 = {
    'Tipo de capas': 'Dense',
    'Tamaño capa de entrada': 64,
    'Número de capas ocultas': 2,
    'Función de activación': "ReLU",
    'MSE': mse_test_value,
    'MAE': mae_test_value,
    'R2': r2_score(Y[test],y_test_pred)
}
datos_2 = pd.DataFrame([datos_2])
df_2 = pd.concat([df_2, datos_2], ignore_index=True)

In [ ]:
seed = 7
np.random.seed(seed)

test = np.random.choice(X_train.shape[0], size=int(X_train.shape[0]*0.2), replace=False)
mse_vals = []
mae_vals = []

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
for train, val in kfold.split(X, Y):
    model = Sequential()
    model.add(Dense(64, input_dim=12, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    model.fit(X_train[train], Y[train], epochs=10,verbose=0)
    mse_aux_value, mae_aux_value=model.evaluate(X_train[val], Y[val], verbose=0)
    mse_vals.append(mse_aux_value)
    mae_vals.append(mae_aux_value)

mse_val_value = np.mean(mse_vals)
mae_val_value = np.mean(mae_vals)

y_test_pred = model.predict(X_train[test])
mse_test_value, mae_test_value=model.evaluate(X_train[test], Y[test], verbose=0)

41/41 [==============================] - 0s 5ms/step


In [ ]:
datos_2 = {
    'Tipo de capas': 'Dense',
    'Tamaño capa de entrada': 64,
    'Número de capas ocultas': 3,
    'Función de activación': "ReLU",
    'MSE': mse_test_value,
    'MAE': mae_test_value,
    'R2': r2_score(Y[test],y_test_pred)
}
datos_2 = pd.DataFrame([datos_2])
df_2 = pd.concat([df_2, datos_2], ignore_index=True)

In [ ]:
seed = 7
np.random.seed(seed)

test = np.random.choice(X_train.shape[0], size=int(X_train.shape[0]*0.2), replace=False)
mse_vals = []
mae_vals = []

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
for train, val in kfold.split(X, Y):
    model = Sequential()
    model.add(Dense(64, input_dim=12, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    model.fit(X_train[train], Y[train], epochs=10,verbose=0)
    mse_aux_value, mae_aux_value=model.evaluate(X_train[val], Y[val], verbose=0)
    mse_vals.append(mse_aux_value)
    mae_vals.append(mae_aux_value)

mse_val_value = np.mean(mse_vals)
mae_val_value = np.mean(mae_vals)

y_test_pred = model.predict(X_train[test])
mse_test_value, mae_test_value=model.evaluate(X_train[test], Y[test], verbose=0)

41/41 [==============================] - 0s 4ms/step


In [ ]:
datos_2 = {
    'Tipo de capas': 'Dense',
    'Tamaño capa de entrada': 64,
    'Número de capas ocultas': 4,
    'Función de activación': "ReLU",
    'MSE': mse_test_value,
    'MAE': mae_test_value,
    'R2': r2_score(Y[test],y_test_pred)
}
datos_2 = pd.DataFrame([datos_2])
df_2 = pd.concat([df_2, datos_2], ignore_index=True)
print(df_2)

  Tipo de capas  Tamaño capa de entrada  Número de capas ocultas  \
0         Dense                      64                        1   
1         Dense                      64                        2   
2         Dense                      64                        3   
3         Dense                      64                        4   

  Función de activación       MSE       MAE        R2  
0                  ReLU  0.509352  0.570927  0.318943  
1                  ReLU  0.467152  0.531323  0.375368  
2                  ReLU  0.427917  0.508226  0.427830  
3                  ReLU  0.431062  0.520329  0.423624  


De igual manera que en el caso de las épocas, a mayor número de capas ocultas el rendimiento mejora considerablemente, la métrica R2 aumenta, mientras que la función de pérdidas y el error absoluto medio disminuyen.

### Pruebas con distinto número de neuronas

In [ ]:
seed = 7
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

neuronas = [64, 76, 88, 100, 128]
df_3 = pd.DataFrame(columns=[])

for i in range(len(neuronas)):
    for train, val in kfold.split(X, Y):
        model = Sequential()
        model.add(Dense(neuronas[i], input_dim=12, activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
        model.fit(X_train[train], Y[train], epochs=10, verbose=0)
        mse_aux_value, mae_aux_value=model.evaluate(X_train[val], Y[val], verbose=0)
        mse_vals.append(mse_aux_value)
        mae_vals.append(mae_aux_value)

    mse_val_value = np.mean(mse_vals)
    mae_val_value = np.mean(mae_vals)

    y_test_pred = model.predict(X_train[test])
    mse_test_value, mae_test_value=model.evaluate(X_train[test], Y[test], verbose=0)

    datos_3 = {
    'Tipo de capas': 'Dense',
    'Tamaño capa de entrada': 64,
    'Número de neuronas': neuronas[i],
    'Función de activación': "ReLU",
    'MSE': mse_test_value,
    'MAE': mae_test_value,
    'R2': r2_score(Y[test],y_test_pred)
    }

    datos_3 = pd.DataFrame([datos_3])
    df_3 = pd.concat([df_3, datos_3], ignore_index=True)

print(df_3)

41/41 [==============================] - 0s 3ms/step
  Tipo de capas  Tamaño capa de entrada  Número de neuronas  \
0         Dense                      64                  64   
1         Dense                      64                  76   
2         Dense                      64                  88   
3         Dense                      64                 100   
4         Dense                      64                 128   

  Función de activación       MSE       MAE        R2  
0                  ReLU  0.473111  0.532757  0.367401  
1                  ReLU  0.469998  0.528612  0.371563  
2                  ReLU  0.469766  0.526443  0.371872  
3                  ReLU  0.476765  0.526775  0.362515  
4                  ReLU  0.460104  0.524475  0.384793  


Conforme se aumenta el número de neuronas el rendimiento aumenta igual que en las pruebas anteriores.

##### Recordad que hay que tener en cuenta que las redes tengan el menor número de neuronas, por varios motivos, ¿Cuáles son estos motivos?, ¿Cuándo se considera que una red sobrepasa el tamaño adecuado y deja de ser conveniente?

Reducir el número de neuronas en una red neuronal es fundamental por varios motivos. El primero de ellos es mejorar la eficiencia computacional, luego también sirve para prevenir el sobreajuste y facilitar la interpretabilidad del modelo. Esto requiere un equilibrio entre la complejidad del problema y los recursos disponibles para así obtener un buen rendimiento y aprovechar los beneficios de los motivos mencionados anteriormente.

En mi opinión, se considera que una red neuronal ha sobrepasado el tamaño adecuado y deja de ser conveniente cuando comienza a mostrar signos de sobreajuste o cuando su rendimiento no mejora significativamente con un aumento en el número de neuronas o capas.

### Pruebas con diferentes parámetros de optimización

In [ ]:
from keras.optimizers import SGD, RMSprop

seed = 7
np.random.seed(seed)

test = np.random.choice(X_train.shape[0], size=int(X_train.shape[0]*0.2), replace=False)
mse_vals = []
mae_vals = []
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
df_4 = pd.DataFrame(columns=[])

rms_values = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1]

for i in range(len(rms_values)):
    rmsprop=RMSprop(learning_rate=rms_values[i])
    for train, val in kfold.split(X, Y):
        model = Sequential()
        model.add(Dense(128, input_dim=12, activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer = rmsprop, loss='mse', metrics=['mae'])
        model.fit(X_train[train], Y[train], epochs=10,verbose=0)
        mse_aux_value, mae_aux_value=model.evaluate(X_train[val], Y[val], verbose=0)
        mse_vals.append(mse_aux_value)
        mae_vals.append(mae_aux_value)

    mse_val_value = np.mean(mse_vals)
    mae_val_value = np.mean(mae_vals)

    y_test_pred = model.predict(X_train[test])
    mse_test_value, mae_test_value=model.evaluate(X_train[test], Y[test], verbose=0)

    datos_4 = {
    'Tipo de capas': 'Dense',
    'Tamaño capa de entrada': 128,
    'Optimizador': 'RMSprop',
    'Learning rate': rms_values[i],
    'Función de activación': "ReLU",
    'MSE': mse_test_value,
    'MAE': mae_test_value,
    'R2': r2_score(Y[test],y_test_pred)
    }

    datos_4 = pd.DataFrame([datos_4])
    df_4 = pd.concat([df_4, datos_4], ignore_index=True)

print(df_4)

41/41 [==============================] - 0s 3ms/step
  Tipo de capas  Tamaño capa de entrada Optimizador  Learning rate  \
0         Dense                     128     RMSprop        0.00001   
1         Dense                     128     RMSprop        0.00010   
2         Dense                     128     RMSprop        0.00100   
3         Dense                     128     RMSprop        0.01000   
4         Dense                     128     RMSprop        0.10000   
5         Dense                     128     RMSprop        1.00000   

  Función de activación        MSE       MAE         R2  
0                  ReLU  27.736919  5.171472 -36.087203  
1                  ReLU   1.336101  0.911541  -0.786508  
2                  ReLU   0.456515  0.526572   0.389591  
3                  ReLU   0.632622  0.618646   0.154117  
4                  ReLU   0.562461  0.594587   0.247930  
5                  ReLU   1.727011  1.123256  -1.309197  


Vemos que por debajo de un learning rate de 0.001 la medida de rendimiento R2 sale negativa, lo mismo ocurre con learning rates mayores de 0.1. Lo óptimo sería usar valores por el rango de 0.001 a 0.1.

In [ ]:
seed = 7
np.random.seed(seed)

test = np.random.choice(X_train.shape[0], size=int(X_train.shape[0]*0.2), replace=False)
mse_vals = []
mae_vals = []
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

sgd_values = [0.0001, 0.001, 0.01, 0.1]

for i in range(len(sgd_values)):
    sgd=SGD(learning_rate=sgd_values[i])
    for train, val in kfold.split(X, Y):
        model = Sequential()
        model.add(Dense(128, input_dim=12, activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer= sgd, loss='mse', metrics=['mae'])
        model.fit(X_train[train], Y[train], epochs=10,verbose=0)
        mse_aux_value, mae_aux_value=model.evaluate(X_train[val], Y[val], verbose=0)
        mse_vals.append(mse_aux_value)
        mae_vals.append(mae_aux_value)

    mse_val_value = np.mean(mse_vals)
    mae_val_value = np.mean(mae_vals)

    y_test_pred = model.predict(X_train[test])
    mse_test_value, mae_test_value=model.evaluate(X_train[test], Y[test], verbose=0)

    datos_4 = {
    'Tipo de capas': 'Dense',
    'Tamaño capa de entrada': 128,
    'Optimizador': 'SGD',
    'Learning rate': sgd_values[i],
    'Función de activación': "ReLU",
    'MSE': mse_test_value,
    'MAE': mae_test_value,
    'R2': r2_score(Y[test],y_test_pred)
    }

    datos_4 = pd.DataFrame([datos_4])
    df_4 = pd.concat([df_4, datos_4], ignore_index=True)

print(df_4)

41/41 [==============================] - 0s 4ms/step
  Tipo de capas  Tamaño capa de entrada Optimizador  Learning rate  \
0         Dense                     128     RMSprop        0.00001   
1         Dense                     128     RMSprop        0.00010   
2         Dense                     128     RMSprop        0.00100   
3         Dense                     128     RMSprop        0.01000   
4         Dense                     128     RMSprop        0.10000   
5         Dense                     128     RMSprop        1.00000   
6         Dense                     128         SGD        0.00010   
7         Dense                     128         SGD        0.00100   
8         Dense                     128         SGD        0.01000   
9         Dense                     128         SGD        0.10000   

  Función de activación        MSE       MAE         R2  
0                  ReLU  27.736919  5.171472 -36.087203  
1                  ReLU   1.336101  0.911541  -0.786508  
2 

Similar al caso anterior, para este tipo de optimizador por debajo de learning rates de 0.01 la medida de rendimiento  R2 es negativa. La diferencia está en que este optimizador trabaja en learning rates hasta 10 veces mayores que el rmsprop.

##### ¿Cuál sería para ti la mejor configuración del modelo que se adapta al problema? ¿Por qué?

De todas las configuraciones que se han probado en este guión, la que mejor se adapta al problema es aquella que no tiene capas ocultas y cuya capa de entrada tiene 64 neuronas. Esta configuración usa el optimizador rmsprop con parámetros por defecto entranada durante 40 épocas.